In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import gzip
import json

import matplotlib
import pickle

import torch
import altair as alt

In [8]:
with open("../../../tempo-ql-sepsis-models/model_checkpoints/task_metrics.pkl", "rb") as file:
    df = pd.DataFrame(pickle.load(file))
df = df[(df['loss_weighting'] == 'unusualness') & (df['noise_factor'] == 0.0) & (df['mask_prob'] == 0.0) & 
        (df['past_value_reconstruction'] == True) & (df['first_value_reconstruction'] == False)]
df['rmse'] = df['mse'] ** 0.5
df['Dataset'] = df['dataset'].replace({'mimiciv': 'MIMIC-IV (Held-out Set)', 'eicu': 'eICU', 'ehrshot': 'EHRSHOT'})

In [9]:
# task_df = df[df['task'] == 'VasopressorNeed']
# print(task_df['auroc'].max())

dataset_order = ['MIMIC-IV (Held-out Set)', 'eICU', 'EHRSHOT']

vaso_chart = alt.Chart(df[df['task'] == 'VasopressorNeed'])
morta_chart = alt.Chart(df[df['task'] == 'Mortality'])
((morta_chart.mark_point(filled=True, size=40).encode(
    y=alt.Y('Dataset:N', axis=alt.Axis(title=None), sort=dataset_order),
    x=alt.X('mean(auroc):Q', scale=alt.Scale(zero=False), title=None),
    yOffset='architecture:N',
    color='architecture:N',
) + morta_chart.mark_errorbar().encode(
    y=alt.Y('Dataset:N', axis=alt.Axis(title=None), sort=dataset_order),
    x='auroc:Q',
    yOffset='architecture:N',
    color='architecture:N',
)).properties(width=250, height=150, title='Mortality') & ((vaso_chart.mark_point(filled=True, size=40).encode(
    y=alt.Y('Dataset:N', axis=alt.Axis(title=None), sort=dataset_order),
    x=alt.X('mean(auroc):Q', scale=alt.Scale(zero=False)),
    yOffset='architecture:N',
    color='architecture:N',
) + vaso_chart.mark_errorbar().encode(
    y=alt.Y('Dataset:N', axis=alt.Axis(title=None), sort=dataset_order),
    x='auroc:Q',
    yOffset='architecture:N',
    color='architecture:N',
)).properties(width=250, height=150, title='Vasopressor Need in 12 h'))).resolve_scale(x='shared').configure_axis(
    labelFontSize=15,
    titleFontSize=18
).configure_title(fontSize=18).configure_legend(titleFontSize=18,
labelFontSize=18)

alt.VConcatChart(...)

In [127]:
sofa_chart = alt.Chart(df[df['task'] == 'SOFA'])
disch_chart = alt.Chart(df[df['task'] == 'DischargeTime'])
((disch_chart.mark_point(filled=True, size=40).encode(
    y=alt.Y('Dataset:N', axis=alt.Axis(title=None), sort=dataset_order),
    x=alt.X('mean(rmse):Q', scale=alt.Scale(zero=False), title=None),
    yOffset='architecture:N',
    color='architecture:N',
) + disch_chart.mark_errorbar().encode(
    y=alt.Y('Dataset:N', axis=alt.Axis(title=None), sort=dataset_order),
    x='rmse:Q',
    yOffset='architecture:N',
    color='architecture:N',
)).properties(width=250, height=150, title='Hours to Discharge') & ((sofa_chart.mark_point(filled=True, size=40).encode(
    y=alt.Y('Dataset:N', axis=alt.Axis(title=None), sort=dataset_order),
    x=alt.X('mean(rmse):Q', scale=alt.Scale(zero=False)),
    yOffset='architecture:N',
    color='architecture:N',
) + sofa_chart.mark_errorbar().encode(
    y=alt.Y('Dataset:N', axis=alt.Axis(title=None), sort=dataset_order),
    x='rmse:Q',
    yOffset='architecture:N',
    color='architecture:N',
)).properties(width=250, height=150, title='Change in SOFA Score 12 h'))).configure_axis(
    labelFontSize=15,
    titleFontSize=18
).configure_title(fontSize=18).configure_legend(titleFontSize=18,
labelFontSize=18)

alt.VConcatChart(...)

In [14]:
disch_chart = alt.Chart(df[(df['task'] == 'DischargeTime') & (df['architecture'] == 'transformer')])
(disch_chart.mark_point(filled=True, size=40).encode(
    y=alt.Y('Dataset:N', axis=alt.Axis(title=None), sort=dataset_order),
    x=alt.X('mean(rmse):Q', scale=alt.Scale(zero=False), title=None),
) + disch_chart.mark_errorbar().encode(
    y=alt.Y('Dataset:N', axis=alt.Axis(title=None), sort=dataset_order),
    x='rmse:Q',
)).properties(width=250, height=100, title='Time to Discharge')

alt.LayerChart(...)